# 1. Module Import and Management Scenario: You are developing a complex Python project with multiple modules. To manage the project effectively, you need to import and use various modules. Additionally, you want to organize your code using namespaces and avoid naming conflicts.

Design a Python program that demonstrates the following:

1. Import multiple modules within your project.

2. Use the import statement to access functions, classes, and variables from imported modules.

3. Create your custom module and use it in your main program. 

4. Handle naming conflicts and ensure proper namespacing.

5. Implement error handling for missing modules or incorrect module usage.

In [1]:
#importing multiple modules
import square_of_number
import math
#Handle naming conflicts and ensure proper namespacing.
from even_or_odd import even_or_odd as number_check 
try:
    
    n = int(input("Enter a number to find square root : "))
    print(f"square root of number : {n} is {math.sqrt(n)}")
    number = 6
    print(f"{number} is {number_check(number)}")

    n = 5
    obj = square_of_number.Class_for_square()
    print(f"module name is : {square_of_number.module_name}")
    print(f"square of number : {n} by calling function {square_of_number.square_of_number(5)}")
    print(f"square of number : {n} by calling method inside calss is : {obj.square_from_class_method(5)}")
except ImportError:
    print("Some modules are missing!")
except AttributeError:
    print("Error in accessing a function/class/variable from a module!")

Enter a number to find square root : 8
square root of number : 8 is 2.8284271247461903
6 is even
module name is : square of a number
square of number : 5 by calling function 25
square of number : 5 by calling method inside calss is : square of number : 5 from method inside class: 25


# 3. Module Dependency Resolution Scenario: You are developing a Python application that relies on third-party packages. dependencies and ensuring compatibility is crucial for your project's success.

Design a Python program that demonstrates the following:

1. Use a requirements.txt file to specify project dependencies.

2. Automatically install all project dependencies from the requirements.txt file.

3. Ensure that the versions of installed packages are compatible.

4. Implement error handling for dependency resolution and installation.

In [6]:
import subprocess
import sys

def install_requirements():
    try:
        # Ensure pip is available
        subprocess.check_call([sys.executable, "-m", "ensurepip"])

        # Upgrade pip
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])

        # Install project dependencies
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])

        print("All dependencies installed successfully!")

    except subprocess.CalledProcessError as e:
        print(f"Error occurred while installing dependencies: {e}")
        sys.exit(1)


install_requirements()



All dependencies installed successfully!


# 1. Implement Inventory Management in Python with MySQL

a) Inventory management, a critical element of the supply chain, is the tracking of inventory from manufacturers to warehouses and from these facilities to a point of sale. The goal of inventory management is to have the right products in the right place at the right time.

b) The required Database is Inventory, and the required Tables are Purchases, Sales and Inventory

c) Note: Apply your thoughts to demonstrate the DB Operation in Python.

In [4]:
import mysql.connector

def connect_to_db():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="1234",
        database="Inventory"
    )
    return connection

def add_purchase(product_name, quantity, purchase_date):
    conn = connect_to_db()
    cursor = conn.cursor()
    
    # Add to Purchases table
    cursor.execute("INSERT INTO Purchases (product_name, quantity, purchase_date) VALUES (%s, %s, %s)", (product_name, quantity, purchase_date))
    
    # Update InventoryStock table
    cursor.execute("SELECT quantity FROM InventoryStock WHERE product_name = %s", (product_name,))
    result = cursor.fetchone()
    if result:
        new_quantity = result[0] + quantity
        cursor.execute("UPDATE InventoryStock SET quantity = %s WHERE product_name = %s", (new_quantity, product_name))
    else:
        cursor.execute("INSERT INTO InventoryStock (product_name, quantity) VALUES (%s, %s)", (product_name, quantity))
    
    conn.commit()
    cursor.close()
    conn.close()

def add_sale(product_name, quantity, sale_date):
    conn = connect_to_db()
    cursor = conn.cursor()
    
    # Add to Sales table
    cursor.execute("INSERT INTO Sales (product_name, quantity, sale_date) VALUES (%s, %s, %s)", (product_name, quantity, sale_date))
    
    # Update InventoryStock table
    cursor.execute("SELECT quantity FROM InventoryStock WHERE product_name = %s", (product_name,))
    result = cursor.fetchone()
    if result and result[0] >= quantity:
        new_quantity = result[0] - quantity
        cursor.execute("UPDATE InventoryStock SET quantity = %s WHERE product_name = %s", (new_quantity, product_name))
    else:
        print("Not enough stock!")
        return
    
    conn.commit()
    cursor.close()
    conn.close()

def get_inventory():
    conn = connect_to_db()
    cursor = conn.cursor()
    
    cursor.execute("SELECT * FROM InventoryStock")
    results = cursor.fetchall()
    
    for product in results:
        print(f"Product: {product[0]}, Quantity: {product[1]}")
    
    cursor.close()
    conn.close()

# Test our system
add_purchase("Laptop", 10, "2023-09-17")
add_sale("Laptop", 2, "2023-09-17")
get_inventory()


Product: Laptop, Quantity: 24


# 2. Customer Order Processing

Scenario: You are building a customer order processing system for an e-commerce company. The system needs to interact with a MySQL database to store customer orders, products, and order details.

1. Design a MySQL database schema for the order processing system, including tables for customers, products, and orders.

2. Write a Python program that connects to the database and allows customers to place

new orders.

3. Implement a feature that calculates the total cost of an order and updates product quantities in the database.

4. How would you handle cases where a product is no longer available when a customer

places an order?

5. Develop a function to generate order reports for the company's finance department.

In [9]:
import mysql.connector

def connect_to_db():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="1234",
        database="OrderProcessing"
    )
    return connection

def place_order(customer_id, products):
    print("Starting order placement...")
    conn = connect_to_db()
    cursor = conn.cursor()

    total_cost = 0
    for product_id, quantity in products.items():
        cursor.execute("SELECT price, quantity FROM Products WHERE product_id = %s", (product_id,))
        product = cursor.fetchone()
        if not product:
            print(f"Product with ID {product_id} not found!")
            return
        price, available_quantity = product
        if available_quantity < quantity:
            print(f"Only {available_quantity} units of product {product_id} are available.")
            return
        total_cost += price * quantity

    cursor.execute("INSERT INTO Orders (customer_id, order_date, total_price) VALUES (%s, CURDATE(), %s)", (customer_id, total_cost))
    order_id = cursor.lastrowid

    for product_id, quantity in products.items():
        cursor.execute("INSERT INTO OrderDetails (order_id, product_id, quantity) VALUES (%s, %s, %s)", (order_id, product_id, quantity))
        cursor.execute("UPDATE Products SET quantity = quantity - %s WHERE product_id = %s", (quantity, product_id))

    conn.commit()
    cursor.close()
    conn.close()
    print("Order placed successfully!")

def generate_order_report():
    print("Generating order report...")
    conn = connect_to_db()
    cursor = conn.cursor()

    cursor.execute("SELECT order_id, order_date, total_price, name, email FROM Orders JOIN Customers ON Orders.customer_id = Customers.customer_id ORDER BY order_date DESC")
    orders = cursor.fetchall()

    for order in orders:
        order_id, order_date, total_price, customer_name, customer_email = order
        print(f"Order ID: {order_id}, Date: {order_date}, Total Price: {total_price}, Customer Name: {customer_name}, Email: {customer_email}")
        cursor.execute("SELECT product_name, OrderDetails.quantity, price FROM OrderDetails JOIN Products ON OrderDetails.product_id = Products.product_id WHERE order_id = %s", (order_id,))
        order_details = cursor.fetchall()
        for detail in order_details:
            product_name, quantity, price = detail
            print(f"   Product: {product_name}, Quantity: {quantity}, Price: {price}")
        print("-------------")

    cursor.close()
    conn.close()

# Testing the functions:
try:
    place_order(1, {1: 2, 2: 1})
    generate_order_report()
except Exception as e:
    print(f"An error occurred: {e}")


Starting order placement...
Product with ID 1 not found!
Generating order report...


# 3. You are tasked with developing a Python program that connects to a MySQL database, retrieves data from a table, performs some operations on the data, and updates the database with the modified data. Please write Python code to accomplish this task.

Instructions:

1. Assume that you have a MySQL database server running with the following details:

L. Host: localhost

i. Port: 3306

ill. Username: your username

iv. Password: your password

v. Database Name: your database

vi. Table Name: your table

vil. The table has the following columns: id (int), name (varchar),

quantity (int).

2. Your Python program should:

1. Connect to the MySQL database.

H. Retrieve all records from the your table table.

. Calculate the total quantity of all records retrieved.

iv. Update the quantity column of each record by doubling its value..

v. Commit the changes to the database.

vi. Close the database connection.

3. Handle any potential errors that may occur during the database connection and data manipulation, such as connection failures or SQL errors.

4 Provide comments in your code to explain each step.

In [11]:
import mysql.connector

def connect_to_database():
    """Connect to the MySQL database and return the connection."""
    try:
        connection = mysql.connector.connect(
            host="localhost",
            port=3306,
            user="root",
            password="1234",
            database="shirts_db"
        )
        return connection
    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None

def retrieve_and_update_data():
    # Connect to the database
    conn = connect_to_database()
    if conn is None:
        return
    
    cursor = conn.cursor()

    try:
        # Retrieve all records
        cursor.execute("SELECT id, quantity FROM shirts")
        records = cursor.fetchall()

        # Calculate the total quantity of all records
        total_quantity = sum([record[1] for record in records])
        print(f"Total quantity before update: {total_quantity}")

        # Update the quantity column of each record by doubling its value
        for record in records:
            updated_quantity = record[1] * 2
            cursor.execute("UPDATE your_table SET quantity = %s WHERE id = %s", (updated_quantity, record[0]))

        # Commit the changes to the database
        conn.commit()
        
        # Print total quantity after update for verification
        cursor.execute("SELECT SUM(quantity) FROM shirts")
        new_total = cursor.fetchone()[0]
        print(f"Total quantity after update: {new_total}")

    except mysql.connector.Error as err:
        print(f"SQL error: {err}")
    
    finally:
        # Close the cursor and the connection
        cursor.close()
        conn.close()

# Execute the function
retrieve_and_update_data()


SQL error: 1054 (42S22): Unknown column 'id' in 'field list'


# 4. You are developing an employee management system for a company. The database should store employee information, including name, salary, department, and hire date. Managers should be able to view and update employee details.

1. Design the database schema for the employee management system.

2. Write Python code to connect to the database and retrieve a list of employees in a specific department.

3. Implement a feature to update an employee's salary.

In [12]:
import mysql.connector

def connect_to_database():
    """Connect to the MySQL database and return the connection."""
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="1234",
        database="EmployeeManagement"
    )
    return connection

def get_employees_by_department(department_name):
    """Retrieve a list of employees in a specific department."""
    conn = connect_to_database()
    cursor = conn.cursor()

    cursor.execute("SELECT name, salary, hire_date FROM Employees WHERE department = %s", (department_name,))
    records = cursor.fetchall()

    for record in records:
        print(f"Name: {record[0]}, Salary: {record[1]}, Hire Date: {record[2]}")

    cursor.close()
    conn.close()
def update_employee_salary(employee_id, new_salary):
    """Update the salary of a specific employee."""
    conn = connect_to_database()
    cursor = conn.cursor()

    cursor.execute("UPDATE Employees SET salary = %s WHERE employee_id = %s", (new_salary, employee_id))
    conn.commit()

    print(f"Updated employee ID {employee_id}'s salary to {new_salary}.")

    cursor.close()
    conn.close()

# Get employees from a specific department:
get_employees_by_department("IT")

# Update an employee's salary:
update_employee_salary(2, 75000)



Updated employee ID 2's salary to 75000.
